<a href="https://colab.research.google.com/github/m-bashari-m/sample-DNN-projects/blob/main/4.3_cat_vs_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import tensorflow as tf
import os

_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir =  "/root/.keras/datasets/cats_and_dogs_filtered/train"
validation_dir = "/root/.keras/datasets/cats_and_dogs_filtered/validation"

In [51]:
train_ds = tf.data.Dataset.list_files(str(train_dir+'/*/*'))
val_ds = tf.data.Dataset.list_files(str(validation_dir+'/*/*'))

In [52]:
for path in train_ds.take(5):
  print(path)

tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.250.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/cats/cat.576.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.449.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.906.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/cats/cat.653.jpg', shape=(), dtype=string)


In [53]:
def prepare_labeled_data(path):
  class_names = tf.constant(['cat', 'dog'])

  categorical_label = tf.strings.split(path, '/')[-1]
  categorical_label = tf.strings.split(categorical_label, '.')[0]
  numerical_label = tf.cast(categorical_label == class_names[0], tf.int8) # cat=1, dog=0

  image = tf.image.decode_image( tf.io.read_file(path), expand_animations=False )
  
  return image,  numerical_label

In [54]:
def augmentation(img):
  img = tf.image.resize_with_crop_or_pad(img, 260, 260)
  img = tf.image.random_crop(img, size=[256, 256, 3])
  img = tf.image.random_brightness(img, max_delta=.2)
  img = tf.image.random_flip_left_right(img)
  return img

In [55]:
def normalization(img):
  img = tf.image.convert_image_dtype(img, tf.float32)
  return img / 255

In [56]:
def train_preprocessing(path):
  img, label = prepare_labeled_data(path)
  img = augmentation(img)
  img = normalization(img)
  return img, label

In [57]:
def test_preprocessing(path):
  img, label = prepare_labeled_data(path)
  img = normalization(img)
  img = tf.image.resize(img, [256, 256])
  return img, label

In [58]:
BATCH_SIZE=64
AUTOTUNE = tf.data.AUTOTUNE
BUFFER_SIZE=1000

In [59]:
train_ds = train_ds.map(train_preprocessing).shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
val_ds = val_ds.map(test_preprocessing).batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [60]:
for img, lbl in train_ds.take(3):
  print(img.shape)

(64, 256, 256, 3)
(64, 256, 256, 3)
(64, 256, 256, 3)


In [61]:
class NormalizationAndActivation(tf.keras.layers.Layer):
  def __init__(self, activation='relu', **kwargs):
    super(NormalizationAndActivation, self).__init__(**kwargs)
    self.activation = tf.keras.activations.get(activation)
    self.batch_norm = tf.keras.layers.BatchNormalization()

  def call(self, X):
    return self.batch_norm(self.activation(X))


In [62]:
class SequentialBlock(tf.keras.layers.Layer):
  def __init__(self, filters, has_dropout=False, dropout_rate=.2, **kwargs):
    super(SequentialBlock, self).__init__(**kwargs)
    self.conv_1 = tf.keras.layers.Conv2D(filters, (3,3), padding='same')
    self.bn_activation_1 = NormalizationAndActivation()
    self.conv_2 = tf.keras.layers.Conv2D(filters, (3,3), strides=(2,2), padding='same')
    self.bn_activation_2 = NormalizationAndActivation()
    self.max_pool = tf.keras.layers.MaxPooling2D(2)

    self.has_dropout = has_dropout
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, X):
    X = self.conv_1(X)
    X = self.bn_activation_1(X)
    X = self.conv_2(X)
    X = self.bn_activation_2(X)
    X = self.max_pool(X)

    if self.has_dropout:
      X = self.dropout(X)

    return X

In [63]:
class CatVsDog(tf.keras.models.Model):
  def __init__(self, activation='relu', **kwargs):
    super(CatVsDog, self).__init__(**kwargs)

    self.blocks = [
                   SequentialBlock(64),
                   SequentialBlock(128),
                   SequentialBlock(512, has_dropout=True),
                   SequentialBlock(1024, has_dropout=True)]

    self.model_layers = [
                   tf.keras.layers.GlobalAveragePooling2D(name='avg_pool_1_26'),
                   tf.keras.layers.Flatten(),
                   tf.keras.layers.Dense(200, name='dense_1_27'),
                   tf.keras.layers.Dropout(.5),
                   tf.keras.layers.Dense(1, name='classifier'),
                   tf.keras.layers.Activation('sigmoid')]

  def call(self, X):
    for block in self.blocks:
      X = block(X)

    for layer in self.model_layers:
      X = layer(X)

    return X

In [64]:
import os

path = '/content/checkpoints'
try:
    os.mkdir(path)
except OSError as error:
    pass

checkpoint = tf.keras.callbacks.ModelCheckpoint(path, save_wights_only=True)


In [65]:
model = CatVsDog()

model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Nadam(),
              metrics=['accuracy'])

In [66]:
history = model.fit(train_ds, 
                    epochs=25,
                    batch_size=64,
                    validation_data=val_ds)

Epoch 1/25
32/32 [==============================] - 96s 948ms/step - loss: 3.0748 - accuracy: 0.5360 - val_loss: 0.8734 - val_accuracy: 0.5000
Epoch 2/25
32/32 [==============================] - 33s 934ms/step - loss: 1.7984 - accuracy: 0.5595 - val_loss: 1.0838 - val_accuracy: 0.5000
Epoch 3/25
32/32 [==============================] - 34s 939ms/step - loss: 1.6273 - accuracy: 0.5300 - val_loss: 0.7632 - val_accuracy: 0.5000
Epoch 4/25
32/32 [==============================] - 33s 936ms/step - loss: 1.3087 - accuracy: 0.5705 - val_loss: 0.7068 - val_accuracy: 0.5000
Epoch 5/25
32/32 [==============================] - 34s 938ms/step - loss: 1.3106 - accuracy: 0.5580 - val_loss: 0.9731 - val_accuracy: 0.5000
Epoch 6/25
32/32 [==============================] - 33s 936ms/step - loss: 1.3275 - accuracy: 0.5845 - val_loss: 1.7628 - val_accuracy: 0.5000
Epoch 7/25
32/32 [==============================] - 34s 937ms/step - loss: 1.0776 - accuracy: 0.6095 - val_loss: 1.0117 - val_accuracy: 0.5000